In [5]:
import pandas as pd
import os

def analyser_structure_fichier():
    print("=== ANALYSE DE LA STRUCTURE DU FICHIER ===")
    
    chemin_fichier = 'hackaton_projet_W3D5/DATASET/plats_populaires_ingredients.csv'
    
    try:
        # Lecture du fichier avec différents séparateurs
        print("\nTentative de lecture avec différents séparateurs...")
        
        # Essai avec point-virgule
        df_semicolon = pd.read_csv(chemin_fichier, sep=';', nrows=5)
        print("\nLecture avec point-virgule :")
        print("Colonnes trouvées :", df_semicolon.columns.tolist())
        print("\nPremières lignes :")
        print(df_semicolon.head(2))
        
        # Essai avec virgule
        df_comma = pd.read_csv(chemin_fichier, sep=',', nrows=5)
        print("\nLecture avec virgule :")
        print("Colonnes trouvées :", df_comma.columns.tolist())
        print("\nPremières lignes :")
        print(df_comma.head(2))
        
        # Affichage des informations détaillées
        print("\nInformations détaillées du DataFrame :")
        print(df_semicolon.info())
        
        # Vérification des valeurs nulles
        print("\nVérification des valeurs manquantes :")
        print(df_semicolon.isnull().sum())
        
    except Exception as e:
        print(f"Erreur lors de l'analyse : {e}")

# Exécution
if __name__ == "__main__":
    analyser_structure_fichier()


=== ANALYSE DE LA STRUCTURE DU FICHIER ===

Tentative de lecture avec différents séparateurs...

Lecture avec point-virgule :
Colonnes trouvées : ['Plat,Pays,Catégorie,Ingrédients,Nombre_Ingrédients']

Premières lignes :
  Plat,Pays,Catégorie,Ingrédients,Nombre_Ingrédients
0  Beef Brisket Pot Roast,ÉTATS-UNIS,Viande,Beef ...
1  Big Mac,ÉTATS-UNIS,Fast-food,Minced Beef (400g...

Lecture avec virgule :
Colonnes trouvées : ['Plat', 'Pays', 'Catégorie', 'Ingrédients', 'Nombre_Ingrédients']

Premières lignes :
                     Plat        Pays  Catégorie  \
0  Beef Brisket Pot Roast  ÉTATS-UNIS     Viande   
1                 Big Mac  ÉTATS-UNIS  Fast-food   

                                         Ingrédients  Nombre_Ingrédients  
0  Beef Brisket (4-5 pound) | Salt (Dash) | Onion...                  11  
1  Minced Beef (400g) | Olive Oil (2 tbs) | Sesam...                  14  

Informations détaillées du DataFrame :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4

In [38]:
import os
import pandas as pd
import requests
import time
from typing import Dict, List, Tuple

class NutrientAnalyzer:
    def __init__(self):
        self.API_KEY = "V47bTvrwdE63sv3FPzRPbyo30fxLmg4cJnc6tWol"
        self.base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
        
        # Nutriments à analyser avec les termes exacts de l'API USDA
        self.nutriments_cibles = {
            'Protein': 'Protéines (g)',
            'Total lipid (fat)': 'Lipides (g)',
            'Carbohydrate, by difference': 'Glucides (g)',
            'Energy': 'Calories (kcal)',
            'Fiber, total dietary': 'Fibres (g)',
            'Sugars, total': 'Sucres (g)',
            'Sodium, Na': 'Sodium (mg)'
        }
        
        # Portions standard pour les ingrédients
        portions_standard = {
            # Viandes et volailles
            'chicken breast': 170,    # g par poitrine
            'chicken legs': 230,      # g par cuisse
            'chicken thighs': 160,    # g par haut de cuisse
            'chicken': 150,           # g par portion standard
            'beef': 200,             # g par portion
            'beef brisket': 450,     # g par portion
            'pork': 200,             # g par portion
            'pork chops': 200,       # g par côtelette
            'bacon': 30,             # g par tranche
            'lamb': 175,             # g par portion
            'lamb leg': 750,         # g par gigot
            'lamb mince': 125,       # g par portion
            'minced beef': 100,      # g par portion
            'duck': 200,             # g par portion
            'veal': 300,             # g par jarret
            'salmon': 150,           # g par portion
            'prawns': 30,            # g par pièce
            'king prawns': 30,       # g par pièce
            'raw king prawns': 30,   # g par pièce
            'clams': 100,            # g par portion
            
            # Légumes
            'onion': 150,            # g par oignon moyen
            'garlic': 5,             # g par gousse
            'garlic clove': 5,       # g par gousse
            'carrot': 120,           # g par carotte
            'potato': 150,           # g par pomme de terre
            'tomato': 120,           # g par tomate
            'mushroom': 15,          # g par champignon
            'shiitake mushrooms': 10, # g par champignon
            'pepper': 120,           # g par poivron
            'yellow pepper': 120,     # g par poivron
            'red pepper': 120,        # g par poivron
            'lettuce': 50,           # g par portion
            'courgette': 200,        # g par courgette
            'zucchini': 200,         # g par courgette
            'aubergine': 250,        # g par aubergine
            'spring onion': 15,      # g par oignon vert
            'scallion': 15,          # g par oignon vert
            'celery': 40,            # g par branche
            'fennel': 200,           # g par bulbe
            'bean sprouts': 50,      # g par portion
            'peas': 80,              # g par portion
            'green beans': 100,      # g par portion
            'chinese broccoli': 100, # g par portion
            'cabbage': 150,          # g par portion
            'asparagus': 100,        # g par portion
            'spinach': 150,          # g par portion
            'broccoli': 200,         # g par tête
            
            # Fromages et produits laitiers
            'cheese': 30,            # g par tranche
            'cheddar cheese': 30,    # g par tranche
            'mozzarella': 125,       # g par boule
            'parmesan': 25,          # g par portion
            'feta': 50,              # g par portion
            'cream': 240,            # ml par tasse
            'double cream': 240,     # ml par tasse
            'heavy cream': 240,      # ml par tasse
            'greek yogurt': 170,     # g par portion
            'full fat yogurt': 170,  # g par portion
            'butter': 14,            # g par cuillère à soupe
            'vegan butter': 14,      # g par cuillère à soupe
            'ghee': 15,              # g par cuillère à soupe
            
            # Féculents et céréales
            'rice': 75,              # g cru par portion
            'basmati rice': 75,      # g cru par portion
            'sushi rice': 75,        # g cru par portion
            'pasta': 85,             # g cru par portion
            'lasagne sheets': 50,    # g par feuille
            'noodles': 85,           # g cru par portion
            'udon noodles': 85,      # g cru par portion
            'rice stick noodles': 85, # g cru par portion
            'flour': 120,            # g par tasse
            'plain flour': 120,      # g par tasse
            'bread': 30,             # g par tranche
            'pita bread': 60,        # g par pain
            'burger bun': 60,        # g par pain
            'bun': 60,               # g par pain
            'breadcrumbs': 30,       # g par portion
            'couscous': 75,          # g cru par portion
            'bulgur wheat': 75,      # g cru par portion
            'quinoa': 75,            # g cru par portion
            
            # Herbes et épices
            'basil': 5,              # g par portion
            'parsley': 5,            # g par portion
            'thyme': 2,              # g par branche
            'rosemary': 2,           # g par branche
            'oregano': 2,            # g par cuillère à soupe
            'marjoram': 2,           # g par cuillère à soupe
            'ginger': 15,            # g par morceau
            'coriander': 5,          # g par portion
            'cilantro': 5,           # g par portion
            'mint': 5,               # g par portion
            'dill': 5,               # g par portion
            'bay leaf': 1,           # g par feuille
            'saffron': 0.5,          # g par portion
            'paprika': 5,            # g par cuillère à café
            'cumin': 5,              # g par cuillère à café
            'cumin seeds': 5,        # g par cuillère à café
            'turmeric': 5,           # g par cuillère à café
            'garam masala': 5,       # g par cuillère à café
            'chilli powder': 5,      # g par cuillère à café
            'allspice': 5,           # g par cuillère à café
            'fennel seeds': 5,       # g par cuillère à café
            'cardamom': 1,           # g par gousse
            'cloves': 0.5,           # g par clou
            'cinnamon': 5,           # g par bâton
            
            # Sauces et condiments
            'soy sauce': 15,         # ml par cuillère à soupe
            'dark soy sauce': 15,    # ml par cuillère à soupe
            'fish sauce': 15,        # ml par cuillère à soupe
            'thai fish sauce': 15,   # ml par cuillère à soupe
            'oyster sauce': 15,      # ml par cuillère à soupe
            'worcestershire sauce': 15, # ml par cuillère à soupe
            'tomato puree': 15,      # g par cuillère à soupe
            'passata': 100,          # g par portion
            'mustard': 15,           # g par cuillère à soupe
            'mayonnaise': 14,        # g par cuillère à soupe
            'tomato ketchup': 15,    # g par cuillère à soupe
            'harissa spice': 5,      # g par cuillère à café
            'thai green curry paste': 15, # g par cuillère à soupe
            
            # Liquides et alcools
            'olive oil': 15,         # ml par cuillère à soupe
            'vegetable oil': 15,     # ml par cuillère à soupe
            'sesame seed oil': 15,   # ml par cuillère à soupe
            'peanut oil': 15,        # ml par cuillère à soupe
            'sunflower oil': 15,     # ml par cuillère à soupe
            'goose fat': 15,         # g par cuillère à soupe
            'stock': 240,            # ml par tasse
            'chicken stock': 240,    # ml par tasse
            'vegetable stock': 240,  # ml par tasse
            'water': 240,            # ml par tasse
            'white wine': 240,       # ml par tasse
            'rice wine': 15,         # ml par cuillère à soupe
            'sake': 15,              # ml par cuillère à soupe
            'mirin': 15,             # ml par cuillère à soupe
            
            # Autres ingrédients
            'egg': 50,               # g par œuf
            'egg white': 30,         # g par blanc
            'tofu': 100,             # g par portion
            'sugar': 4,              # g par cuillère à café
            'brown sugar': 5,        # g par cuillère à café
            'caster sugar': 4,       # g par cuillère à café
            'honey': 21,             # g par cuillère à soupe
            'yeast': 7,              # g par sachet
            'dried apricots': 20,    # g par portion
            'chickpeas': 120,        # g par portion
            'cashew nuts': 30,       # g par portion
            'black olives': 15,      # g par portion
            'sesame seed': 5,        # g par cuillère à café
            'khus khus': 5,          # g par cuillère à café
        }
        
        # Conversions de base
        self.conversions = {
            # Volumes
            'cup': 240,              # ml
            'cups': 240,             # ml
            'tsp': 5,                # ml (cuillère à café)
            'teaspoon': 5,           # ml
            'tbsp': 15,              # ml (cuillère à soupe)
            'tablespoon': 15,        # ml
            'tbs': 15,               # ml
            'fluid oz': 30,          # ml
            'fl oz': 30,             # ml
            'pint': 473,             # ml
            'quart': 946,            # ml
            'gallon': 3785,          # ml
            'ml': 1,                 # ml
            'l': 1000,               # ml
            
            # Poids
            'oz': 28.35,             # g
            'ounce': 28.35,          # g
            'lb': 453.6,             # g
            'pound': 453.6,          # g
            'kg': 1000,              # g
            'g': 1,                  # g
            
            # Portions spécifiques
            'slice': 30,             # g (pour pain, fromage)
            'piece': 1,              # unité
            'clove': 5,              # g (pour l'ail)
            'sprig': 2,              # g (pour herbes fraîches)
            'bunch': 30,             # g (pour herbes)
            'dash': 0.5,             # ml/g
            'pinch': 0.25,           # ml/g
            'handful': 30,           # g
            'large': 1.5,            # multiplicateur
            'small': 0.75,           # multiplicateur
            'medium': 1,             # multiplicateur
        }
        
        # Chemins des fichiers
        self.chemin_base = "/Users/ludovicveltz/Documents/Bootcamp_GENAI_2025/Crashcourse/hackaton_projet_W3D5/DATASET"
        self.chemin_fichier = os.path.join(self.chemin_base, "plats_populaires_ingredients.csv")
        self.chemin_sortie = os.path.join(self.chemin_base, "plats_populaires_nutriments_complet.csv")


    def extraire_ingredients_et_quantites(self, ingredients_str: str) -> List[Tuple[str, float, str]]:
        """Extrait les ingrédients avec leurs quantités et unités"""
        if pd.isna(ingredients_str) or ingredients_str == 'Non disponible':
            return []
        
        ingredients_list = []
        for ing in ingredients_str.split('|'):
            ing = ing.strip()
            if '(' in ing and ')' in ing:
                try:
                    nom = ing.split('(')[0].strip()
                    quantite_str = ing.split('(')[1].split(')')[0].strip()
                    qte, unite = self.normaliser_quantite(quantite_str, nom)
                    ingredients_list.append((nom, qte, unite))
                except Exception as e:
                    print(f"Erreur lors du traitement de l'ingrédient {ing}: {e}")
                    continue
            else:
                for ing_key, portion in self.portions_standard.items():
                    if ing_key in ing.lower():
                        ingredients_list.append((ing, portion, 'g'))
                        break
                else:
                    ingredients_list.append((ing, 1, 'portion'))
        return ingredients_list

    def normaliser_quantite(self, quantite_str: str, ingredient: str) -> Tuple[float, str]:
        """Normalise les quantités en unités standard"""
        # Gestion des cas spéciaux et descriptions
        special_cases = {
            'dash': 0.5,
            'pinch': 0.25,
            'chopped': 5.0,
            'garnish': 5.0,
            'to serve': 5.0,
            'to taste': 1.0,
            'leaves': 2.0
        }
        
        # Si la chaîne est vide ou contient un cas spécial
        if not quantite_str or any(case in quantite_str.lower() for case in special_cases):
            for case, value in special_cases.items():
                if case in str(quantite_str).lower():
                    return value, 'g'
            
            # Portions standard par défaut selon l'ingrédient
            for ing_key, portion in self.portions_standard.items():
                if ing_key in ingredient.lower():
                    return portion, 'g'
            return 5.0, 'g'

        quantite_str = quantite_str.lower().strip()

        # Définition des unités préférées par type d'ingrédient
        ingredient_units = {
            'oil': 'ml',
            'stock': 'ml',
            'vinegar': 'ml',
            'sauce': 'ml',
            'milk': 'ml',
            'cream': 'ml',
            'juice': 'ml',
            'water': 'ml',
            'wine': 'ml',
            'default': 'g'
        }

        # Limites maximales par type d'ingrédient
        limites = {
            'beef': 500,       # g
            'chicken': 300,    # g
            'fish': 300,      # g
            'garlic': 30,      # g
            'onion': 300,      # g
            'cheese': 200,     # g
            'salt': 10,        # g
            'pepper': 5,       # g
            'spice': 10,       # g pour les épices
            'oil': 100,        # ml
            'milk': 500,       # ml
            'flour': 300,      # g
            'sugar': 200,      # g
            'butter': 100,     # g
            'stock': 500,      # ml
            'sauce': 100,      # ml
            'vegetable': 300,  # g
            'default': 300     # g/ml
        }

        try:
            # Extraction et conversion du nombre
            if '/' in quantite_str:
                parts = quantite_str.split()
                nombre = 0
                for part in parts:
                    if '/' in part:
                        num, denom = map(float, part.split('/'))
                        nombre += num / denom
                    elif part.replace('.', '').isdigit():
                        nombre += float(part)
            else:
                nombre = float(''.join([c for c in quantite_str.split()[0] if c.isdigit() or c == '.']))

            # Détermination de l'unité préférée
            unite_preferee = 'g'
            for ing_type, unite in ingredient_units.items():
                if ing_type in ingredient.lower():
                    unite_preferee = unite
                    break

            # Application des limites et conversion
            for ing_type, limite in limites.items():
                if ing_type in ingredient.lower():
                    if nombre > limite:
                        print(f"Ajustement quantité pour {ingredient}: {nombre} -> {limite}")
                        nombre = limite
                    break

            # Conversion selon l'unité trouvée dans la chaîne
            for unite, conversion in self.conversions.items():
                if unite in quantite_str:
                    if unite in ['cup', 'cups', 'tsp', 'tbsp', 'tbs', 'fluid oz', 'fl oz', 'pint', 'quart', 'gallon', 'ml', 'l']:
                        return nombre * conversion, unite_preferee
                    return nombre * conversion, unite_preferee

            # Si pas d'unité trouvée, utiliser l'unité préférée
            return nombre, unite_preferee

        except Exception as e:
            print(f"Erreur de conversion pour {quantite_str}: {e}")
            # Valeur par défaut selon le type d'ingrédient
            for ing_type, limite in limites.items():
                if ing_type in ingredient.lower():
                    return limite/2, ingredient_units.get(ing_type, 'g')
            return 5.0, 'g'





    def rechercher_nutriments(self, ingredient: str, quantite: float, unite: str) -> Dict:
        """Recherche les nutriments pour un ingrédient donné via l'API USDA"""
        params = {
            'api_key': self.API_KEY,
            'query': ingredient,
            'pageSize': 1,
            'dataType': ['Foundation', 'Survey (FNDDS)']
        }
        
        try:
            response = requests.get(self.base_url, params=params)
            if response.status_code == 200:
                data = response.json()
                if data.get('foods'):
                    nutrients = data['foods'][0]['foodNutrients']
                    nutriments = {}
                    nutrients_dict = {n.get('nutrientName'): n for n in nutrients}
                    
                    for api_name, fr_name in self.nutriments_cibles.items():
                        if api_name in nutrients_dict:
                            valeur = nutrients_dict[api_name].get('value', 0)
                            if unite in ['g', 'ml']:
                                valeur = valeur * (quantite / 100)
                            else:
                                valeur = valeur * quantite
                            nutriments[api_name] = valeur
                        else:
                            nutriments[api_name] = 0
                    return nutriments
            return {api_name: 0 for api_name in self.nutriments_cibles.keys()}
        except Exception as e:
            print(f"Erreur pour {ingredient}: {e}")
            return {api_name: 0 for api_name in self.nutriments_cibles.keys()}

    def analyser_plats(self):
        """Analyse nutritionnelle de tous les plats"""
        try:
            df = pd.read_csv(self.chemin_fichier, sep=',')
            print(f"Dataset chargé avec succès : {len(df)} plats")
            
            resultats = []
            total_plats = len(df)
            
            for index, row in df.iterrows():
                plat = row['Plat'].strip()
                print(f"\nAnalyse du plat {index + 1}/{total_plats} : {plat}")
                
                ingredients = self.extraire_ingredients_et_quantites(row['Ingrédients'])
                nutriments_totaux = {value: 0 for value in self.nutriments_cibles.values()}
                
                for ingredient, quantite, unite in ingredients:
                    print(f"  Analyse de : {ingredient} ({quantite} {unite})")
                    nutriments = self.rechercher_nutriments(ingredient, quantite, unite)
                    
                    for api_name, fr_name in self.nutriments_cibles.items():
                        if api_name in nutriments:
                            nutriments_totaux[fr_name] += nutriments[api_name]
                    
                    time.sleep(1)  # Respect des limites de l'API
                
                resultat = {
                    'Plat': plat,
                    'Pays': row['Pays'].strip(),
                    'Catégorie': row['Catégorie'].strip()
                }
                resultat.update(nutriments_totaux)
                resultats.append(resultat)
                
                # Sauvegarde temporaire tous les 5 plats
                if (index + 1) % 5 == 0:
                    df_temp = pd.DataFrame(resultats)
                    df_temp.to_csv(os.path.join(self.chemin_base, "temp_results.csv"), 
                                 index=False, encoding='utf-8-sig')
            
            # Sauvegarde finale
            df_final = pd.DataFrame(resultats)
            df_final.to_csv(self.chemin_sortie, index=False, encoding='utf-8-sig')
            print("\nAnalyse terminée avec succès!")
            return df_final

        except Exception as e:
            print(f"Erreur lors de l'analyse : {e}")
            return None

if __name__ == "__main__":
    print("=== ANALYSE NUTRITIONNELLE DES PLATS TRADITIONNELS ===")
    analyzer = NutrientAnalyzer()
    df_resultat = analyzer.analyser_plats()


=== ANALYSE NUTRITIONNELLE DES PLATS TRADITIONNELS ===
Dataset chargé avec succès : 23 plats

Analyse du plat 1/23 : Couscous
Erreur de conversion pour handful: could not convert string to float: ''
  Analyse de : Olive Oil (15.0 ml)
  Analyse de : Onion (5.0 g)
  Analyse de : Chicken Breast (200.0 g)
  Analyse de : Ginger (0.25 g)
  Analyse de : Harissa Spice (2000.0 g)
  Analyse de : Dried Apricots (10.0 g)
  Analyse de : Chickpeas (220.0 g)
  Analyse de : Couscous (200.0 g)
  Analyse de : Chicken Stock (200.0 ml)
  Analyse de : Coriander (5.0 g)

Analyse du plat 2/23 : Pizza
Erreur de conversion pour drizzle: could not convert string to float: ''
Erreur de conversion pour peeled and sliced: could not convert string to float: ''
  Analyse de : Water (150.0 ml)
  Analyse de : Sugar (5.0 g)
  Analyse de : Yeast (15.0 g)
  Analyse de : Plain Flour (225.0 g)
  Analyse de : Salt (7.5 g)
  Analyse de : Olive Oil (50.0 ml)
  Analyse de : Passata (80.0 g)
  Analyse de : Mozzarella (70.0 g)
 